In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import scipy
import gc
import sklearn
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from keras import layers
from keras.layers import Input, Add, Subtract, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.optimizers import Adam, Adagrad, SGD



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../"))

# Any results you write to the current directory are saved as output.


label_conv = ["carpet","concrete","fine_concrete","hard_tiles","hard_tiles_large_space","soft_pvc","soft_tiles","tiled","wood"]



Using TensorFlow backend.


['X_train.csv', 'sample_submission.csv', 'X_test.csv', 'y_train.csv']
['working', 'lib', 'input', 'src']


In [2]:
#TODO 1.1
#Create full output for submissions
#Add convienence functions, plotting, type changes, etc.
#Add competition result evaluation function
#Set logic into classes?
#AUTO PULL SAMPLE DATA SHAPE INTO PROGRAM


#TODO 1.2
#initialize different random seed each time, use mod of time?
#change layer_dims to allow variable layers per unit
#change to new structure
#Add inhibitory functions
#Change to RELU activation
#allow for activations function change?
#check to make sure Y1[x] = Y2[x]

#TODO 1.3
#Harp style connections, angled out for access to outputs

#TODO for LTSM 
#change batch numbers to random number between 0 and total batches


In [3]:
#Format Input Training Data
def retrieve_traindata():
    raw_train = pd.read_csv('../input/X_train.csv')
    raw_train_key = pd.read_csv('../input/y_train.csv')
    rangemax = raw_train.nlargest(1,'series_id')['series_id'].values
    m = rangemax[0]+1

    X_train = np.zeros((m, 1280)).astype(np.float32)
    for i in range(m):
        temp = raw_train.loc[raw_train['series_id'] == i].iloc[:,3:].values
        for j in range(temp.shape[0]):
            X_train[i,(temp.shape[1]*j):(temp.shape[1]*(1+j))] = temp[j]
    
    Y_train = np.zeros((m,2)).astype(np.float32)
    Y_train = raw_train_key.values
    
    return X_train, Y_train

def retrieve_traindata_extra():
    raw_train = pd.read_csv('../input/X_train.csv')
    raw_train_key = pd.read_csv('../input/y_train.csv')
    rangemax = raw_train.nlargest(1,'series_id')['series_id'].values
    m = rangemax[0]+1

    X_train = np.zeros((m*4, 1280)).astype(np.float32)
    for i in range(m):
        temp = raw_train.loc[raw_train['series_id'] == i].iloc[:,3:].values
        #print("Unfiltered: ", temp[1])
        temp_position = temp.copy()
        temp_position[:,4:].fill(0)
        #print("Position: ", temp_position[1])
        temp_velocity = temp.copy()
        temp_velocity[:,:4].fill(0)
        temp_velocity[:,7:].fill(0)
        #print("Velocity: ", temp_velocity[1])
        temp_accel = temp.copy()
        temp_accel[:,:7].fill(0)
        #print("Accel: ", temp_accel[1])
        for j in range(temp.shape[0]):
            X_train[i,(temp.shape[1]*j):(temp.shape[1]*(1+j))] = temp_position[j]
            X_train[(i+m),(temp.shape[1]*j):(temp.shape[1]*(1+j))] = temp_velocity[j]
            X_train[(i+(2*m)),(temp.shape[1]*j):(temp.shape[1]*(1+j))] = temp_accel[j]
            X_train[(i+(3*m)),(temp.shape[1]*j):(temp.shape[1]*(1+j))] = temp[j]
    
    Y_temp = np.zeros((m,2)).astype(np.float32)
    Y_temp = raw_train_key.values
    
    Y_temp2 = np.concatenate((Y_temp, Y_temp), axis=0)
    Y_train = np.concatenate((Y_temp2, Y_temp2), axis=0)
    
    return X_train, Y_train



#Format Input Testing Data
def retrieve_testdata():
    raw_test = pd.read_csv('../input/X_test.csv')
    rangemax = raw_test.nlargest(1,'series_id')['series_id'].values
    m = rangemax[0]+1

    X_test = np.zeros((m, 1280)).astype(np.float32)
    for i in range(m):
        temp = raw_test.loc[raw_test['series_id'] == i].iloc[:,3:].values
        for j in range(temp.shape[0]):
            X_test[i,(temp.shape[1]*j):(temp.shape[1]*(1+j))] = temp[j]
    

    return X_test


#preprocessing
def dataprep_full(X_rough):
    #Modify X input data to normalize the inputs
    m, n_data = X_rough.shape
    X_data = np.zeros((m, n_data*2)).astype(np.float32)
    
    data_neg = np.less(X_rough, 0)
    data_pos = np.greater(X_rough, 0)
    
    neg_vals = np.multiply(X_rough,data_neg)
    X_neg = neg_vals*-1
    
    X_pos = np.multiply(X_rough,data_pos)
    
    for i in range(m):
        pos_scaled = MinMaxScaler(copy=True, feature_range=(0, 1)).fit_transform(X_pos[i].reshape(len(X_pos[i]), 1))
        X_data[i,:n_data] = pos_scaled[:,0]
        neg_scaled = MinMaxScaler(copy=True, feature_range=(0, 1)).fit_transform(X_neg[i].reshape(len(X_pos[i]), 1))
        X_data[i,n_data:] = neg_scaled[:,0]
    
    return X_data


#preprocessing
def dataprep(X_rough):
    #Modify X input data to normalize the inputs
    m, n_data = X_rough.shape
    X_data = np.zeros((m, n_data)).astype(np.float32)
    
    X_data = MinMaxScaler(copy=True, feature_range=(-1, 1)).fit_transform(X_rough)
    
    
    return X_data


#Modify the Y data into a one hot vector
def ydata_to_onehot_full(Y_rough):
    
    #capture both surface identifier as well as the group id information into a one hot vector
    m, y_len = Y_rough.shape
    
    #create one hot of surfaces
    surfaces = Y_rough[:,2].T
    surf_label_enc = LabelEncoder().fit_transform(surfaces)
    onehot_surf = OneHotEncoder(sparse=False).fit_transform(surf_label_enc.reshape(len(surf_label_enc), 1))
    
    #create one hot of group_id's
    groupid = Y_rough[:,1].T
    groupid_label_enc = LabelEncoder().fit_transform(groupid)
    onehot_groupid = OneHotEncoder(sparse=False).fit_transform(groupid_label_enc.reshape(len(groupid_label_enc), 1))
    
    #create empty array of shape (number of samples, number of different group_id's plus number of different surfaces)
    Y_data = np.zeros((m, 73+9)).astype(np.int16)
    
    for i in range(m):
        Y_data[i,:73] = onehot_groupid[i]
        Y_data[i,73:] = onehot_surf[i]
    
    return Y_data


#Modify the Y data into a one hot vector
def ydata_to_onehot(Y_rough):
    
    #capture both surface identifier as well as the group id information into a one hot vector
    m, y_len = Y_rough.shape
    
    #create one hot of surfaces
    surfaces = Y_rough[:,2].T
    
    str_to_int = dict((c, i) for i, c in enumerate(label_conv))
    # integer encode input data
    integer_encoded = [str_to_int[char] for char in surfaces]
    
    # one hot encode
    Y_data = np.zeros((m, 9)).astype(np.int16)
    count = 0
    for value in integer_encoded:
        surf = [0 for _ in range(len(label_conv))]
        surf[value] = 1
        Y_data[count] = surf
        count +=1
    
    return Y_data

def onehot_to_labels(data):
    
    int_to_str = dict((i, c) for i, c in enumerate(label_conv))
    predictions = []
    for i in range(len(data)):
        entry = (i,int_to_str[np.argmax(data[i])])
        predictions.append(entry)
    
    return predictions

In [4]:
X_rough1, Y_rough1 = retrieve_traindata()
print(X_rough1.shape)
print(Y_rough1.shape)
#print(Y_rough1[1])
X_rough2, Y_rough2 = retrieve_traindata_extra()
print(X_rough2.shape)
print(Y_rough2.shape)
#print(Y_rough2[-5:])

(3810, 1280)
(3810, 3)
(15240, 1280)
(15240, 3)


In [5]:
X = dataprep_full(X_rough1)
X2 = dataprep_full(X_rough2)
#print(X[5])
print(X.shape)

(3810, 2560)


In [6]:
Y = ydata_to_onehot_full(Y_rough1)
Y2 = ydata_to_onehot_full(Y_rough2)
print(Y.shape)
#print(Y[1000])

(3810, 82)


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [7]:
def network(input_shape):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    #Input Network
    X = Dense(2560,input_shape=(2560,),activation='relu',kernel_initializer='TruncatedNormal')(X_input)
    X = Dense(2560,activation='relu',kernel_initializer='TruncatedNormal')(X)
    
    #Exictatory Network

    X = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    X = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    #X = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    
    
    #Pass Through Excitatory Weights
    X_shortcut = X
    
    #Inhibitory Network
    X = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    X = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    #X = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    
    #Combine Excite and Inhibit Networks
    X = Subtract()([X_shortcut, X])
    
    #Create Passthrough for Weights
    X_shortFinal = X
    
    #Decision Network
    X = Dense(1000,activation='relu',kernel_initializer='TruncatedNormal')(X)
    X = Dense(1000,activation='relu',kernel_initializer='TruncatedNormal')(X)
    #X = Dense(1000,activation='relu',kernel_initializer='TruncatedNormal')(X)
    X = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    
    #Combine Total Networks
    X = Add()([X, X_shortFinal])
    X = Activation('relu')(X)
    
    #Output 
    X = Dense(82,activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs = X, name='CarCon Net')
    
    return model

    

In [8]:
def network_LTSM(input_shape):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    #Input Network
    X = Dense(2560,input_shape=(2560,),activation='relu',kernel_initializer='TruncatedNormal')(X_input)
    X = Dense(2560,activation='relu',kernel_initializer='TruncatedNormal')(X)
    
    #Exictatory Network
    #Y = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    X = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    
    #X = Subtract()([X, Y])
    #X = Activation('relu')(X)
    
    #Y = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    X = Dense(500,activation='relu',kernel_initializer='TruncatedNormal')(X)
    
    #X = Subtract()([X, Y])
    #X = Activation('relu')(X)
    
    #Pass Through Excitatory Weights
    X_shortcut = X
    
    #Inhibitory Network
    X = Dense(500,activation='relu',kernel_initializer='ones')(X)
    X = Dense(500,activation='relu',kernel_initializer='ones')(X)
    
    #Combine Excite and Inhibit Networks
    X = Subtract()([X_shortcut, X])
    X = Activation('relu')(X)
    
    #Create Passthrough for Weights
    X_shortFinal = X
    
    #Decision Network
    X = Dense(1000,activation='relu',kernel_initializer='zeros')(X)
    X = Dense(1000,activation='relu',kernel_initializer='zeros')(X)
    X = Dense(1000,activation='relu',kernel_initializer='zeros')(X)
    X = Dense(500,activation='relu',kernel_initializer='zeros')(X)
    
    #Combine Total Networks
    X = Add()([X, X_shortFinal])
    X = Activation('relu')(X)
    
    #Output 
    X = Dense(82,activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs = X, name='CarCon Net')
    
    return model

In [9]:
model = network(input_shape=X[0].shape)


Instructions for updating:
Colocations handled automatically by placer.


In [10]:
optimize = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#optimize = Adagrad(lr=0.02, epsilon=.01, decay=0.0)
#optimize = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(optimizer=optimize, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X2, Y2, epochs = 100, batch_size = 50)
model.fit(X, Y, epochs = 100, batch_size = 50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
15240/15240 [==============================] - 6s 408us/step - loss: 7.0057 - acc: 0.0073
Epoch 2/100
15240/15240 [==============================] - 3s 221us/step - loss: 6.1951 - acc: 0.0259
Epoch 3/100
15240/15240 [==============================] - 3s 221us/step - loss: 5.8231 - acc: 0.0436
Epoch 4/100
15240/15240 [==============================] - 3s 221us/step - loss: 5.5882 - acc: 0.0515
Epoch 5/100
15240/15240 [==============================] - 3s 220us/step - loss: 5.3579 - acc: 0.0581
Epoch 6/100
15240/15240 [==============================] - 3s 221us/step - loss: 5.1760 - acc: 0.0670
Epoch 7/100
15240/15240 [==============================] - 3s 220us/step - loss: 4.9306 - acc: 0.0780
Epoch 8/100
15240/15240 [==============================] - 3s 222us/step - loss: 4.7233 - acc: 0.0919
Epoch 9/100
15240/15240 [==============================] - 3s 220us/step - loss: 4.5338 - acc: 0.1012
Epoch 10/100
15240/15240 [========

In [12]:
X_rough_test = retrieve_testdata()

In [13]:
X_rough_test.shape

(3816, 1280)

In [14]:
X_test = dataprep_full(X_rough_test)

In [15]:
X_test.shape

(3816, 2560)

In [16]:
predict_raw = model.predict(X_test)

In [17]:
predict = (predict_raw > .5)
print(predict.shape)

(3816, 82)


In [18]:
#predict_labels = onehot_to_labels(predict[:,73:])
#predict_labels = onehot_to_labels(predict)
#predict_labels = onehot_to_labels(predict_raw)
predict_labels = onehot_to_labels(predict_raw[:,73:])

In [19]:
col_labels = ['series_id', 'surface']
prediction_pd = pd.DataFrame.from_records(predict_labels, columns=col_labels)

In [20]:
prediction_pd.shape

(3816, 2)

In [21]:
prediction_pd.to_csv('X_test_submission10.csv', encoding='utf-8', index=False)

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2560)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 2560)         6556160     input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 2560)         6556160     dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 500)          1280500     dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (D